
## 🧭 Introduction : Prise en main d’un notebook Jupyter

Un **notebook Jupyter** est un document interactif qui combine **texte explicatif** (comme celui que vous lisez), **code exécutable**, et **résultats**. Il est très utilisé en science des données, en apprentissage automatique et en recherche.

### ✅ Objectifs de cette section

* Comprendre le fonctionnement d’un notebook
* Savoir exécuter du code Python
* Se préparer à interagir avec des modèles de langage





### 🔹 Cellules de texte (comme celle-ci)

Cette cellule contient du texte en **Markdown**, un langage de mise en forme simple. Vous pouvez écrire des titres, listes, liens, formules mathématiques, etc.
Double cliquez dessus pour modifier.

### 🔹 Cellules de code

Voici une première cellule de code à exécuter : cliquez dessus, puis faites **Shift + Entrée** (ou le bouton ▶️ en haut).


In [ ]:
# Une cellule Python simple
print("Bienvenue dans le TP sur les modèles de langage ! 🤖")

### 🔸 Variables et calculs en Python

In [ ]:
a = 2
b = 3
somme = a + b
print("La somme de a et b est :", somme)

### 🔸 Utiliser des bibliothèques

En Python, on utilise des bibliothèques pour accéder à des outils puissants. Par exemple :

In [ ]:
import math
print("La racine carrée de 16 est :", math.sqrt(16))

## 📌 Rappel : Comment utiliser ce notebook

* **Shift + Entrée** : exécute une cellule et passe à la suivante
* Vous pouvez **modifier** les cellules et voir comment le résultat change
* Vous pouvez **ajouter** vos propres cellules (clic droit > "Insert cell below")


### 🔄 À vous de jouer !

👉 Modifiez la cellule ci-dessous pour afficher un message personnalisé :



In [ ]:
# À modifier
mon_nom = "Votre prénom"
print(f"Bonjour {mon_nom}, amusez-vous bien avec les LLM ! 🤓")

# TP : Introduction pratique aux modèles NLP et LLM

## Objectifs du TP

Dans ce TP, nous allons explorer :
1. **La représentation du langage naturel** : tokenizers et encodage du texte
2. **Les caractéristiques des modèles GPT** : prédiction de tokens, context window
3. **L'impact du prompting et de la température** sur les sorties du modèle
4. **La diversité des modèles** disponibles sur Hugging Face

---

## Installation des dépendances

Exécutez la cellule suivante pour installer les bibliothèques nécessaires :

In [ ]:
# Installation des bibliothèques
!pip install transformers torch -q

# Imports nécessaires
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    pipeline
)
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques installées et importées avec succès!")

---

## Partie 1 : Représenter le langage naturel - Les Tokenizers

### 1.1 Introduction aux tokenizers

Les tokenizers sont des outils essentiels qui convertissent le texte en tokens (unités de base) que les modèles peuvent traiter.

In [ ]:
# Textes d'exemple pour nos tests
textes_exemples = [
    "Bonjour, comment allez-vous aujourd'hui?",
    "The quick brown fox jumps over the lazy dog.",
    "人工智能正在改变世界。",  # Chinois : "L'IA change le monde"
    "🤖 Les emojis sont-ils bien tokenisés? 🚀",
    "import numpy as np\nprint('Hello World!')"
]

# Fonction utilitaire pour visualiser la tokenisation
def visualiser_tokenisation(tokenizer, texte, nom_tokenizer):
    tokens = tokenizer.tokenize(texte)
    token_ids = tokenizer.encode(texte)
    
    print(f"\n{'='*60}")
    print(f"Tokenizer: {nom_tokenizer}")
    print(f"Texte original: {texte}")
    print(f"Nombre de tokens: {len(tokens)}")
    print(f"Tokens: {tokens[:20]}{'...' if len(tokens) > 20 else ''}")
    print(f"Token IDs: {token_ids[:20]}{'...' if len(token_ids) > 20 else ''}")
    
    return len(tokens)

### 1.2 Comparaison de différents tokenizers

Nous allons comparer plusieurs tokenizers populaires :

In [ ]:
# Chargement de différents tokenizers
tokenizers = {
    "GPT-2": AutoTokenizer.from_pretrained("gpt2"),
    "BERT": AutoTokenizer.from_pretrained("bert-base-uncased"),
    "T5": AutoTokenizer.from_pretrained("t5-small"),
    "CamemBERT (Français)": AutoTokenizer.from_pretrained("camembert-base")
}

# Analyse comparative
resultats = {}
for nom, tokenizer in tokenizers.items():
    print(f"\n\n{'#'*80}\n{nom.upper()}\n{'#'*80}")
    resultats[nom] = []
    
    for texte in textes_exemples:
        try:
            nb_tokens = visualiser_tokenisation(tokenizer, texte, nom)
            resultats[nom].append(nb_tokens)
        except:
            print(f"❌ Erreur avec le texte: {texte}")
            resultats[nom].append(0)

### 📝 Question 1.1

**Observez les résultats ci-dessus. Que remarquez-vous concernant :**
- Le nombre de tokens pour le même texte selon les tokenizers ?
- La gestion des caractères spéciaux et emojis ?
- La tokenisation du code Python ?

**Votre réponse :** *(Double-cliquez pour éditer)*

...

In [ ]:
# Visualisation graphique des résultats
import pandas as pd

df_resultats = pd.DataFrame(resultats, index=[f"Texte {i+1}" for i in range(len(textes_exemples))])

# Graphique
fig, ax = plt.subplots(figsize=(10, 6))
df_resultats.plot(kind='bar', ax=ax)
ax.set_title("Nombre de tokens par tokenizer et par texte")
ax.set_xlabel("Textes")
ax.set_ylabel("Nombre de tokens")
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nRésumé statistique:")
print(df_resultats.describe())

### 1.3 Exploration approfondie : Vocabulaire et tokens spéciaux

In [ ]:
# Explorons le vocabulaire d'un tokenizer
tokenizer_gpt2 = tokenizers["GPT-2"]

print(f"Taille du vocabulaire GPT-2: {tokenizer_gpt2.vocab_size}")
print(f"\nTokens spéciaux:")
print(f"- Token de padding: {tokenizer_gpt2.pad_token}")
print(f"- Token de début: {tokenizer_gpt2.bos_token}")
print(f"- Token de fin: {tokenizer_gpt2.eos_token}")
print(f"- Token inconnu: {tokenizer_gpt2.unk_token}")

# Exemple de décodage
texte_test = "Intelligence artificielle"
tokens_ids = tokenizer_gpt2.encode(texte_test)
texte_decode = tokenizer_gpt2.decode(tokens_ids)

print(f"\nEncodage/Décodage:")
print(f"Texte original: '{texte_test}'")
print(f"Token IDs: {tokens_ids}")
print(f"Texte décodé: '{texte_decode}'")
print(f"Correspondance parfaite: {texte_test == texte_decode}")

### 📝 Exercice 1.1 : Analyse de tokenisation

Modifiez le texte ci-dessous et observez comment la tokenisation change :

In [ ]:
# TODO: Modifiez ce texte pour tester différents cas
mon_texte = "L'intelligence artificielle révolutionne notre quotidien!"

# Testez avec différents tokenizers
for nom, tokenizer in list(tokenizers.items())[:2]:  # On teste avec les 2 premiers
    tokens = tokenizer.tokenize(mon_texte)
    print(f"\n{nom}:")
    print(f"  Tokens ({len(tokens)}): {tokens}")
    
    # Reconstruction token par token
    print("  Reconstruction: ", end="")
    for token in tokens:
        print(f"[{token}]", end="")
    print()

---

## Partie 2 : Comprendre les modèles GPT

### 2.1 Choix et chargement d'un modèle

Voici quelques modèles que vous pouvez explorer sur Hugging Face 🤗

- 🧠 GPT-2 (baseline) : https://huggingface.co/gpt2-medium
- ⚡ DistilGPT-2 (plus rapide, plus petit) : https://huggingface.co/distilgpt2
- 🔬 Phi-2 (Microsoft, très performant pour sa taille) : https://huggingface.co/microsoft/phi-2
- 🦙 TinyLLaMA (modèle compact basé sur LLaMA) : https://huggingface.co/cuongvm/tinyllama-1.1b-chat-v1.0
- 🌪️ Mistral 7B (quantisé, très puissant mais plus gros) : https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

👉 Vous pouvez modifier la variable `model_name` ci-dessous pour tester un autre modèle.

In [ ]:
# Choisir votre modèle ici (il sera utilisé dans toute la suite du tp, n'hesitez pas a venir le changer ici plus tard)
model_name = "gpt2-medium"  # Exemple: "gpt2-medium",  "Qwen/Qwen3-0.6B", "distilgpt2", "microsoft/phi-2", etc.
print(f"🔄 Chargement du modèle {model_name}... (cela peut prendre un moment)")

# Chargement du tokenizer et du modèle
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Token de padding (certaines architectures en ont besoin)
tokenizer.pad_token = tokenizer.eos_token

# Affichage des infos du modèle
print(f"\n✅ Modèle chargé: {model_name}")
print(f"📊 Nombre de paramètres: {model.num_parameters():,}")
try:
    print(f"📏 Taille du contexte maximum: {model.config.max_position_embeddings} tokens")
except:
    print("📏 Taille du contexte maximum: information non disponible")

### 2.2 Génération de texte : Prédiction de tokens

In [ ]:

def generer_texte(prompt, max_length=50, temperature=1.0, num_return_sequences=1):
    """
    Génère du texte à partir d'un prompt
    """
    # Encodage
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    # Génération
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True
        )
    
    # Décodage et affichage
    for i, output in enumerate(outputs):
        texte_genere = tokenizer.decode(output, skip_special_tokens=True)
        print(f"\n📝 Génération {i+1}:")
        print(f"{'='*60}")
        print(texte_genere)

In [ ]:
# Test de génération (lancer plusieurs fois)
prompt_test = "The artificial intelligence is"
print(f"Prompt: '{prompt_test}'\n")
generer_texte(prompt_test, max_length=15, temperature=0.8)

In [ ]:
# Test de génération, essayer avec du texte en francais ou autre langue. Que remarquez-vous?
prompt_test = "l'intelligence artificielle est"
print(f"Prompt: '{prompt_test}'\n")
generer_texte(prompt_test, max_length=15, temperature=0.8)

In [ ]:
# Essayer different sujet. Que remarquez-vous?
prompt_test = "17 - 2 ="
print(f"Prompt: '{prompt_test}'\n")
generer_texte(prompt_test, max_length=15, temperature=0.8)

### 2.3 Importance du bon tokenizer

In [ ]:
# Démonstration : utiliser le mauvais tokenizer
print("🔍 Expérience : Que se passe-t-il si on utilise le mauvais tokenizer?\n")

# Tokenizer correct (GPT-2)
prompt = "France is "
inputs_correct = tokenizer.encode(prompt, return_tensors="pt")

# Tokenizer incorrect (BERT)
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs_incorrect = tokenizer_bert.encode(prompt, return_tensors="pt")

print(f"Prompt: '{prompt}'")
print(f"\nTokens GPT-2 (correct): {tokenizer.tokenize(prompt)}")
print(f"Token IDs GPT-2: {inputs_correct[0].tolist()}")
print(f"\nTokens BERT (incorrect): {tokenizer_bert.tokenize(prompt)}")
print(f"Token IDs BERT: {inputs_incorrect[0].tolist()}")

# Génération avec le bon tokenizer
with torch.no_grad():
    output_correct = model.generate(inputs_correct, max_length=20, temperature=0)
    texte_correct = tokenizer.decode(output_correct[0], skip_special_tokens=True)
    
print(f"\n✅ Génération avec le bon tokenizer: '{texte_correct}'")

# Tentative avec les IDs du mauvais tokenizer (attention aux erreurs!)
try:
    # On va créer des IDs aléatoires pour simuler
    fake_ids = torch.randint(0, tokenizer.vocab_size, (1, len(inputs_incorrect[0])))
    with torch.no_grad():
        output_incorrect = model.generate(fake_ids, max_length=20, temperature=0.1)
        texte_incorrect = prompt + tokenizer.decode(output_incorrect[0], skip_special_tokens=True)[:20]
    print(f"❌ Génération avec mauvais tokenizer: '{texte_incorrect}'")
except Exception as e:
    print(f"❌ Erreur avec le mauvais tokenizer: {str(e)}")

### 📝 Question 2.4

**Pourquoi est-il crucial d'utiliser le tokenizer correspondant au modèle?**

**Votre réponse :** *(Double-cliquez pour éditer)*

...

### 2.5 Limites du contexte (Context Window)

In [ ]:
# Démonstration de la limite de contexte
print(f"🔍 Limite de contexte du modèle: {model.config.n_ctx} tokens\n")

# Créons un texte très long. Essayons un contexte trop long!
texte_court = "Ceci est un exemple. " * 10
texte_long = "Ceci est un exemple. " * 1000

tokens_court = tokenizer.encode(texte_court)
tokens_long = tokenizer.encode(texte_long)

print(f"Texte court: {len(tokens_court)} tokens")
print(f"Texte long: {len(tokens_long)} tokens")
print(f"Dépasse la limite? {len(tokens_long) > model.config.n_ctx}")

# Visualisation de ce qui se passe avec un texte trop long
if len(tokens_long) > model.config.n_ctx:
    print(f"\n⚠️ Le texte long ({len(tokens_long)} tokens) dépasse la limite!")
    print(f"Le modèle ne peut traiter que les {model.config.n_ctx} premiers tokens.")
    
    # Tronquons pour la démonstration
    tokens_tronques = tokens_long[:model.config.n_ctx]
    texte_tronque = tokenizer.decode(tokens_tronques)
    print(f"\nTexte tronqué (début): {texte_tronque[:100]}...")
    print(f"Texte tronqué (fin): ...{texte_tronque[-100:]}")

### 2.6 Date limite d'entraînement

In [ ]:
# Test des connaissances temporelles du modèle
prompts_temporels = [
    "The president of the United States in 2019 was",
    "The COVID-19 pandemic started in",
    "The latest iPhone model in 2023 is",
    "The winner of the 2022 FIFA World Cup was"
]

print("🕐 Test des connaissances temporelles du modèle\n")
print("Note: GPT-2 a été entraîné sur des données jusqu'en 2019\n")

for prompt in prompts_temporels:
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=len(inputs[0]) + 20,
            temperature=0.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    resultat = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Q: {prompt}")
    print(f"R: {resultat}")
    print("-" * 60)

### 📝 Exercice 2.7 : Explorer les limites du modèle

Testez le modèle avec vos propres prompts pour identifier ses limites temporelles :

In [ ]:
# TODO: Ajoutez vos propres prompts pour tester les connaissances du modèle
mes_prompts = [
    "In 2018, the french constitution",
    # Ajoutez vos prompts ici
]

for prompt in mes_prompts:
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=50, temperature=0.3, pad_token_id=tokenizer.eos_token_id)
    print(f"Prompt: {prompt}")
    print(f"Réponse: {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")

---

## Partie 3 : Impact du prompting et de la température

### 3.1 Exploration de la température

In [ ]:
# Fonction pour visualiser l'impact de la température
def explorer_temperature(prompt, temperatures=[0.1, 0.5, 0.8, 1.0, 1.5]):
    print(f"Prompt: '{prompt}'\n")
    
    resultats = {}
    
    for temp in temperatures:
        print(f"\n🌡️ Température = {temp}")
        print("=" * 60)
        
        inputs = tokenizer.encode(prompt, return_tensors="pt")
        generations = []
        
        # Générer plusieurs fois pour voir la variabilité
        for i in range(3):
            with torch.no_grad():
                outputs = model.generate(
    inputs,
                    max_length=50,
                    temperature=temp,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
            
            texte = tokenizer.decode(outputs[0], skip_special_tokens=True)
            generations.append(texte)
            print(f"Essai {i+1}: {texte}")
        
        resultats[temp] = generations
    
    return resultats

# Test avec différentes températures
resultats_temp = explorer_temperature("Une recette de cuisine innovante:")

### 📝 Question 3.1

**Observez l'impact de la température sur les générations:**
- Que se passe-t-il avec une température basse (0.1) ?
- Que se passe-t-il avec une température élevée (1.5) ?
- Quelle température recommanderiez-vous pour des cas d'usage créatifs vs factuels ?

**Votre réponse :** *(Double-cliquez pour éditer)*

...

### 3.2 L'art du prompting

In [ ]:
# Comparaison de différents styles de prompts
tache = "expliquer ce qu'est le machine learning"

prompts_styles = {
    "Basique": "Le machine learning est",
    
    "Avec contexte": "En tant que professeur d'informatique, j'explique à mes étudiants que le machine learning est",
    
    "Avec exemple": "Le machine learning est comme apprendre à faire du vélo. Au début,",
    
    "Structuré": "Définition du Machine Learning:\n1.",
    
    "Conversation": "Étudiant: Qu'est-ce que le machine learning?\nProfesseur: Excellente question! Le machine learning est"
}

print("🎯 Comparaison de différents styles de prompting\n")

for style, prompt in prompts_styles.items():
    print(f"\n{'='*80}")
    print(f"Style: {style}")
    print(f"Prompt: '{prompt}'")
    print("-" * 80)
    
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=100,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True
        )
    
    resultat = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(resultat)

### 📝 Exercice 3.1 : Créez vos propres prompts

Expérimentez avec différents prompts pour une tâche de votre choix :

In [ ]:
# TODO: Créez 3 prompts différents pour accomplir la même tâche
ma_tache = "générer une histoire courte sur un robot"

mes_prompts = {
    "Prompt 1": "Il était une fois un robot qui",
    "Prompt 2": "Journal intime d'un robot, jour 1:",
    "Prompt 3": "Breaking news: Un robot extraordinaire vient de"
}

# Testez vos prompts
temperature_choisie = 0.8  # Modifiez cette valeur

for nom, prompt in mes_prompts.items():
    print(f"\n{nom}: '{prompt}'")
    print("-" * 60)
    
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=80,
            temperature=temperature_choisie,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True
        )
    
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 4 Comparaison de modèles : Français vs Anglais

In [ ]:


"""
🎯 Objectif : Comparer deux modèles pré-entraînés, l’un sur de l’anglais, l’autre sur du français.
"""

from transformers import pipeline

# Modèle entraîné principalement sur de l'anglais (GPT-2)
modele_anglais = pipeline("text-generation", model="gpt2", tokenizer="gpt2")

# Modèle entraîné principalement sur du français (CamemBERT + fine-tuning causal LM)
modele_francais = pipeline("text-generation", model="asi/gpt-fr-cased-base", tokenizer="asi/gpt-fr-cased-base")

# Phrase d'amorçage en français
prompt_fr = "La révolution de l'intelligence artificielle"

# Phrase d'amorçage en anglais
prompt_en = "The revolution of artificial intelligence"

# Génération
print("\n🇫🇷 Modèle français input francais:")
res_fr = modele_francais(prompt_fr, max_length=30, do_sample=True, temperature=0.8)[0]["generated_text"]
print(res_fr)

# Génération
print("\n🇫🇷 Modèle français input anglais:")
res_fr = modele_francais(prompt_en, max_length=30, do_sample=True, temperature=0.8)[0]["generated_text"]
print(res_fr)

print("\n🇬🇧 Modèle anglais input anglais:")
res_en = modele_anglais(prompt_en, max_length=30, do_sample=True, temperature=0.8)[0]["generated_text"]
print(res_en)



print("\n🇬🇧 Modèle anglais input francais:")
res_en = modele_anglais(prompt_fr, max_length=30, do_sample=True, temperature=0.8)[0]["generated_text"]
print(res_en)


- Pourquoi ces modeles ne donnent pas du tout les meme résultats ?

**Votre réponse :** *(Double-cliquez pour éditer)*

...

---

## Partie 5 : La diversité des modèles sur Hugging Face

### 5.1 Pipeline

In [ ]:
# Présentation des différentes tâches disponibles
print("🎯 Exemples de tâches NLP disponibles sur Hugging Face:\n")

taches_exemples = [
    "text-generation",
    "sentiment-analysis", 
    "translation",
    "summarization",
    "question-answering",
    "zero-shot-classification",
    "text2text-generation"
]

for tache in taches_exemples:
    print(f"- {tache}")

### 5.2 Analyse de sentiment

In [ ]:
# Pipeline d'analyse de sentiment
sentiment_analyzer = pipeline("sentiment-analysis")

# Textes à analyser
textes_sentiment = [
    "J'adore ce cours sur le NLP, c'est passionnant!",
    "Cette expérience est vraiment décevante.",
    "Le temps est nuageux aujourd'hui.",
    "This transformer model is absolutely amazing! 🚀",
    "Je ne sais pas trop quoi en penser, c'est mitigé."
]

print("😊😐😢 Analyse de sentiment\n")

resultats_sentiment = sentiment_analyzer(textes_sentiment)

for texte, resultat in zip(textes_sentiment, resultats_sentiment):
    emoji = "😊" if resultat['label'] == 'POSITIVE' else "😢"
    print(f"{emoji} Texte: '{texte}'")
    print(f"   → Sentiment: {resultat['label']} (confiance: {resultat['score']:.2%})")
    print()

### 5.3 Classification zero-shot

In [ ]:
# Classification zero-shot : classifier sans entraînement spécifique
classifier = pipeline("zero-shot-classification")

# Textes à classifier
textes_classification = [
    "Tesla annonce une nouvelle voiture électrique autonome",
    "Le PSG remporte le match 3-0 contre Lyon",
    "Nouvelle découverte scientifique sur les trous noirs",
    "Apple dévoile l'iPhone 15 avec des fonctionnalités révolutionnaires"
]

# Catégories possibles
categories = ["sport", "technologie", "science", "politique", "économie"]

print("🏷️ Classification zero-shot\n")

for texte in textes_classification:
    resultat = classifier(texte, candidate_labels=categories)
    
    print(f"Texte: '{texte}'")
    print(f"Classifications:")
    for label, score in zip(resultat['labels'][:3], resultat['scores'][:3]):
        print(f"  - {label}: {score:.2%}")
    print()

### 5.4 Génération de résumés

In [ ]:
# Pipeline de résumé
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Texte long à résumer
article = """
L'intelligence artificielle (IA) transforme rapidement notre monde. 
Des assistants vocaux dans nos maisons aux systèmes de recommandation sur nos plateformes de streaming, 
l'IA est omniprésente. Les récentes avancées en apprentissage profond, notamment avec les transformers, 
ont permis des progrès spectaculaires dans le traitement du langage naturel. 
Ces modèles peuvent maintenant comprendre et générer du texte avec une qualité impressionnante, 
ouvrant de nouvelles possibilités dans de nombreux domaines comme la traduction automatique, 
la génération de contenu, et l'analyse de sentiment. Cependant, ces avancées soulèvent aussi 
des questions éthiques importantes concernant la vie privée, les biais algorithmiques, 
et l'impact sur l'emploi.
"""

print("📝 Génération de résumé\n")
print("Article original:")
print(article)
print("\nRésumé généré:")

resume = summarizer(article, max_length=80, min_length=30, do_sample=False)
print(resume[0]['summary_text'])

### 5.5 Questions-Réponses

In [ ]:
# Pipeline de questions-réponses
qa_pipeline = pipeline("question-answering")

# Contexte
contexte = """
Hugging Face est une entreprise franco-américaine fondée en 2016 par Clément Delangue, Julien Chaumond et Thomas Wolf.
L'entreprise est surtout connue pour sa bibliothèque Transformers, qui fournit des architectures de pointe 
pour le traitement du langage naturel. La plateforme Hugging Face Hub héberge plus de 100 000 modèles 
pré-entraînés que les développeurs peuvent utiliser gratuitement. L'entreprise a levé 235 millions de dollars 
en 2023, atteignant une valorisation de 4,5 milliards de dollars.
"""

# Questions
questions = [
    "Quand a été fondée Hugging Face?",
    "Qui sont les fondateurs?",
    "Combien de modèles sont hébergés sur le Hub?",
    "Quelle est la valorisation de l'entreprise?"
]

print("❓ Questions-Réponses\n")
print(f"Contexte: {contexte}\n")

for question in questions:
    reponse = qa_pipeline(question=question, context=contexte)
    print(f"Q: {question}")
    print(f"R: {reponse['answer']} (confiance: {reponse['score']:.2%})")
    print()

### 📝 Exercice 4.1 : Explorer d'autres modèles

Choisissez un modèle spécialisé sur Hugging Face et testez-le :

In [ ]:
# TODO: Explorez un autre modèle de votre choix
# Exemples de modèles intéressants:
# - "Helsinki-NLP/opus-mt-fr-en" pour la traduction français→anglais
# - "camembert-base" pour du NLP en français
# - "microsoft/DialoGPT-medium" pour la conversation

# Votre code ici
mon_modele = "Helsinki-NLP/opus-mt-fr-en"  # Changez ce modèle
ma_tache = "translation_fr_to_en"  # Adaptez la tâche

# Créez votre pipeline et testez-le
# ...

---

## Conclusion et points clés à retenir

### 🎯 Ce que nous avons appris :

1. **Tokenisation** :
   - Les tokenizers transforment le texte en unités que les modèles peuvent traiter
   - Différents tokenizers ont différentes stratégies (BPE, WordPiece, etc.)
   - Il est crucial d'utiliser le bon tokenizer avec le bon modèle

2. **Modèles GPT** :
   - Fonctionnent par prédiction du token suivant
   - Ont une limite de contexte (context window)
   - Leurs connaissances sont limitées à leur date d'entraînement

3. **Paramètres de génération** :
   - La température contrôle la créativité/aléatoire
   - Le prompting est un art qui influence grandement les résultats

4. **Écosystème Hugging Face** :
   - Grande variété de modèles pour différentes tâches
   - Pipelines pour une utilisation simplifiée
   - Modèles spécialisés vs modèles généralistes

### 📚 Pour aller plus loin :

- Documentation Hugging Face : https://huggingface.co/docs
- Cours sur les Transformers : https://huggingface.co/course
- Model Hub : https://huggingface.co/models
- Papers with Code : https://paperswithcode.com/